In [147]:
package main

import (
  "fmt"
  "go/ast"
  "go/parser"
  "go/printer"
  "go/token"
   "go/format"
  "log"
  "strings"
  "strconv"
  "sort"
  "bytes"
  "testing"
  "bufio"
  "bytes"
  "encoding/json"
  "errors"
  "io/ioutil"
  "log"
  "os"
  "os/exec"
  "path/filepath"
  "time"
  "golang.org/x/tools/go/ast/astutil"
)

In [2]:
const (
  statPass = "pass"
  statFail = "fail"
  statSkip = "skip"
  statErr  = "error"
)

type testResult struct {
  Name     string `json:"name"`
  Status   string `json:"status"`
  TestCode string `json:"test_code"`
  Message  string `json:"message"`
}

type testReport struct {
  Status  string       `json:"status"`
  Message string       `json:"message,omitempty"`
  Tests   []testResult `json:"tests"`
}

type testLine struct {
  Time    time.Time
  Action  string
  Package string
  Test    string
  Elapsed float64
  Output  string
}


In [3]:
func splitTestName(testName string) (string, string) {                          
  t := strings.Split(testName, "/")                                             
  if 1 == len(t) {                                                              
    return t[0], ""                                                             
  }                                                                             
  return t[0], t[1]                                                             
}                                                                               
                                                                                
func findTestFile(testName string, codePath string) string {                    
  test, _ := splitTestName(testName)                                            
  files, err := ioutil.ReadDir(codePath)                                        
  if err != nil {                                                               
    log.Printf("warning: input_dir '%s' cannot be read: %s", codePath, err)     
    return ""                                                                   
  }                                                                             
  testdef := fmt.Sprintf("func %s", test)                                       
  for _, f := range files {                                                     
    if strings.HasSuffix(f.Name(), "_test.go") {                                
      var code string                                                           
      testpath := filepath.Join(codePath, f.Name())                             
      fmt.Scanln(&code)                                                         
      fh, err := ioutil.ReadFile(testpath)                                      
      if err != nil {                                                           
        log.Printf("warning: test file '%s' read failed: %s", testpath, err)    
      }                                                                         
      if strings.Contains(string(fh), testdef) {                                
        return testpath                                                         
      }                                                                         
    }                                                                           
  }                                                                             
  log.Printf("warning: test %s not found in input_dir '%s'", codePath, test)    
  return ""                                                                     
}                                                                               


In [4]:
func extractTestCode(testName string, testFile string) string {                 
  test, subtest := splitTestName(testName)                                      
  if 0 == len(subtest) {                                                        
    return extractFunc(test, testFile)                                          
  }                                                                             
  subtcode := extractSub(test, subtest, testFile)                               
  if 0 == len(subtcode) {                                                       
    return extractFunc(test, testFile)                                          
  }                                                                             
  return subtcode                                                               
}                                                                               
                                                                                
func extractFunc(testName string, testFile string) string {                     
  fset := token.NewFileSet()                                                    
  ppc := parser.ParseComments                                                   
  if file, err := parser.ParseFile(fset, testFile, nil, ppc); err == nil {      
    for _, d := range file.Decls {                                              
      if f, ok := d.(*ast.FuncDecl); ok && f.Name.Name == testName {            
        fun := &printer.CommentedNode{Node: f, Comments: file.Comments}         
        var buf bytes.Buffer                                                    
        printer.Fprint(&buf, fset, fun)                                         
        return buf.String()                                                     
      }                                                                         
    }                                                                           
  } else {                                                                      
    log.Printf(                                                                 
      "warning: '%s' not parsed from '%s': %s", testName, testFile, err,        
    )                                                                           
  }                                                                             
  return ""                                                                     
}                                                                               
                                                                                
func extractSub(test string, sub string, file string) string {                  
  return sub                                                                    
}

In [5]:
func getStructure(lines bytes.Buffer, input_dir string) *testReport {
  report := &testReport{
    Status: statPass,
    Tests:  nil,
  }
  defer func() {
    if report.Tests == nil {
      report.Tests = []testResult{}
    }
  }()

  tests, err := buildTests(lines, input_dir)
  if err != nil {
    report.Status = statErr
    report.Message = err.Error()
    return report
  }
  for _, test := range tests {
    if test == nil {
      // just to be sure we dont get a nil pointer exception
      continue
    }
    if test.Status == statErr {
      report.Status = statErr
    }
    if test.Status == statSkip {
      report.Status = statErr
    }
    if report.Status == statPass && test.Status == statFail {
      report.Status = statFail
    }

    report.Tests = append(report.Tests, *test)
  }

  return report
}
func buildTests(lines bytes.Buffer, input_dir string) (map[string]*testResult, error) {
  var (
    tests       = map[string]*testResult{}
    testFileMap = make(map[string]string)
    failMsg     [][]byte
  )

  scanner := bufio.NewScanner(&lines)
  for scanner.Scan() {
    lineBytes := scanner.Bytes()
    var line testLine

    switch {
    case len(lineBytes) == 0:
      continue
    case !bytes.HasPrefix(lineBytes, []byte{'{'}):
      // if the line is not a json, we need to collect the lines to gather why `go test --json` failed
      failMsg = append(failMsg, lineBytes)
      continue
    }

    if err := json.Unmarshal(lineBytes, &line); err != nil {
      log.Println(err)
      continue
    }

    if line.Test == "" {
      continue
    }

    switch line.Action {
    case "run":
      tf, cached := testFileMap[line.Test]
      if !cached {
        tf = findTestFile(input_dir, line.Test)
        testFileMap[line.Test] = tf
      }
      tc := extractTestCode(line.Test, tf)
      if len(tc) > 0 {
        tests[line.Test] = &testResult{
          Name:     line.Test,
          TestCode: tc,
          Status:   statSkip,
        }
      }
    case "output":
      tests[line.Test].Message += "\n" + line.Output
    case statFail:
      tests[line.Test].Status = statFail
    case statPass:
      tests[line.Test].Status = statPass
    }
  }
  if len(failMsg) != 0 {
    return nil, errors.New(string(bytes.Join(failMsg, []byte{'\n'})))
  }
  return tests, nil
}

In [6]:
//const homedir string = "/home/ekingery/"
const homedir string = "/Users/ekingery/"
input_dir := homedir + "dev/exercism/go-test-runner/testdata/concept/conditionals"
output_dir := homedir + "dev/exercism/go-test-runner/outdir"
goExe, err := exec.LookPath("go")
var stdout, stderr bytes.Buffer

  testCmd := &exec.Cmd{
    Dir:    input_dir,
    Path:   goExe,
    Args:   []string{goExe, "test", "--json", "."},
    Stdout: &stdout,
    Stderr: &stderr,
  }
  if err := testCmd.Run(); err != nil {                                         
    if exitError, ok := err.(*exec.ExitError); ok {                             
      if 1 == exitError.ExitCode() {                                            
        // `go test` returns 1 when tests fail                                  
        // The test runner should continue and return 0 in this case            
        log.Printf(                                                             
          "warning: ignoring exit code 1 from '%s'", testCmd.String(),          
        )                                                                       
      } else {                                                                  
        log.Fatalf("'%s' failed with exit code %d: %s",                         
          testCmd.String(), exitError.ExitCode(), err)                          
      }                                                                         
    }                                                                           
  }      
fmt.Println(&stdout)
//output := getStructure(stdout, input_dir)
//bts, err := json.MarshalIndent(output, "", "\t")


{"Time":"2020-12-11T21:43:45.090566-06:00","Action":"run","Package":"conditionals","Test":"TestNonSubtest"}
{"Time":"2020-12-11T21:43:45.092257-06:00","Action":"output","Package":"conditionals","Test":"TestNonSubtest","Output":"=== RUN   TestNonSubtest\n"}
{"Time":"2020-12-11T21:43:45.092293-06:00","Action":"output","Package":"conditionals","Test":"TestNonSubtest","Output":"the whole block\n"}
{"Time":"2020-12-11T21:43:45.092299-06:00","Action":"output","Package":"conditionals","Test":"TestNonSubtest","Output":"should be returned\n"}
{"Time":"2020-12-11T21:43:45.092731-06:00","Action":"output","Package":"conditionals","Test":"TestNonSubtest","Output":"--- PASS: TestNonSubtest (0.00s)\n"}
{"Time":"2020-12-11T21:43:45.092751-06:00","Action":"pass","Package":"conditionals","Test":"TestNonSubtest","Elapsed":0}
{"Time":"2020-12-11T21:43:45.093172-06:00","Action":"run","Package":"conditionals","Test":"TestParseCard"}
{"Time":"2020-12-11T21:43:45.093183-06:00","Action":"output","Package":"con

14331 <nil>

In [7]:
testName := "TestParseCard"
testFile := homedir + "dev/exercism/go-test-runner/testdata/concept/conditionals/conditionals_test.go"
testCodeSrc := extractFunc(testName, testFile)
fmt.Println(testCodeSrc)

func TestParseCard(t *testing.T) {
	tests := []struct {
		name	string	`json:"name"`
		card	string	`json:"card"`
		want	int	`json:"want"`
	}{
		{
			name:	"parse ace",
			card:	"ace",
			want:	11,
		},
		{
			name:	"parse two",
			card:	"two",
			want:	2,
		},
		{
			name:	"parse three",
			card:	"three",
			want:	3,
		},
		{
			name:	"parse four",
			card:	"four",
			want:	4,
		},
		{
			name:	"parse five",
			card:	"five",
			want:	5,
		},
		{
			name:	"parse six",
			card:	"six",
			want:	6,
		},
		{
			name:	"parse seven",
			card:	"seven",
			want:	7,
		},
		{
			name:	"parse eight",
			card:	"eight",
			want:	8,
		},
		{
			name:	"parse nine",
			card:	"nine",
			want:	9,
		},
		{
			name:	"parse ten",
			card:	"ten",
			want:	10,
		},
		{
			name:	"parse jack",
			card:	"jack",
			want:	10,
		},
		{
			name:	"parse queen",
			card:	"queen",
			want:	10,
		},
		{
			name:	"parse king",
			card:	"king",
			want:	10,
		},
	}
	for _, tt := range tests {
		t.Run(tt.name, func(t *testi

1140 <nil>

In [8]:
fset := token.NewFileSet()
f, err := parser.ParseFile(fset, "", "package main\n" + testCodeSrc, parser.ParseComments)
if err != nil {
    log.Fatal(err)
}
fexp, err := parser.ParseExpr(testCodeSrc)

In [9]:
fmt.Printf("%T, %s\n--\n", f, f)

fmt.Printf("%T, %s", fexp, fexp)


*ast.File, &{%!s(*ast.CommentGroup=<nil>) %!s(token.Pos=1) main [%!s(*ast.FuncDecl=&{<nil> <nil> 0xc000830d00 0xc0007cc000 0xc000833680})] scope 0xc0008384b0 {
	func TestParseCard
}
 [] [testing string string int testing ParseCard] []}
--
*ast.FuncLit, &{%!s(*ast.FuncType=&{1 0xc000833890 <nil>}) %!s(*ast.BlockStmt=&{34 [0xc000837dc0 0xc0008614a0] 1139})}

118 <nil>

In [10]:
var tdAssgn ast.AssignStmt
var tdNode ast.Node
var tda ast.AssignStmt
var tdn ast.Node

ast.Inspect(fexp, func(n ast.Node) bool {
    switch x := n.(type) {
    case *ast.BasicLit:
        if token.STRING == x.Kind && "\"parse queen\"" == x.Value{
            fmt.Printf("bl %s:\t%s - %s\n", fset.Position(n.Pos()), x.Kind, x.Value)
            tdAssgn = tda
            tdNode = tdn
            return false
        }
    case *ast.AssignStmt:
        fmt.Printf("asgn %s: %s - %s\n", fset.Position(n.Pos()), x.Lhs, x.Rhs)
        tda = *x
        tdn = n
		/*case *ast.Ident:
            if x != nil && "tests" == x.Name {
                fmt.Printf("id %s: %s\n", fset.Position(n.Pos()), x.Name)
            } */
	}
	return true
})
LHE := tdAssgn.Lhs[0]
fmt.Printf("%T, %s", LHE, LHE)

//fmt.Printf("tdata name: %s: %s - %s\n", fset.Position(tdNode.Pos()), tdAssgn.Lhs, tdAssgn.Rhs)
//fmt.Printf("asgn %s: %s - %s\n", fset.Position(tdNode.Pos()), tdAssgn.Lhs, tdAssgn.Rhs)
//fmt.Printf("asgn %s: %s - %s\n", fset.Position(tdNode.Pos()), tdAssgn.TokPos, tdAssgn.Tok)

asgn 2:24: [tests] - [%!s(*ast.CompositeLit=&{0xc000833920 140 [0xc000837700 0xc000837780 0xc000837800 0xc0008378c0 0xc000837940 0xc0008379c0 0xc000837a40 0xc000837ac0 0xc000837b40 0xc000837bc0 0xc000837c40 0xc000837cc0 0xc000837d40] 941 false})]
bl 63:1:	STRING - "parse queen"
asgn 75:32: [got] - [%!s(*ast.CallExpr=&{0xc0007cd0e0 1031 [0xc0007cd140] 0 1039})]
*ast.Ident, tests

17 <nil>

In [11]:
//ast.Print(nil, fexp)
//var tdAssgn ast.AssignStmt
type storedNodes struct {
    subtest_name  ast.Node
    test_assign   ast.Node
    loop_varname  string
} 

var tdNode ast.Node
var tdPNode ast.Node
var tda ast.AssignStmt
var tdn ast.Node


astutil.Apply(fexp, func(cr *astutil.Cursor) bool {
    n := cr.Node()
    //n := cr.Parent()
    switch x := n.(type) {
    case *ast.BasicLit:
        if token.STRING == x.Kind && "\"parse queen\"" == x.Value{
            fmt.Printf("bl %s:\t%s - %s\n", fset.Position(n.Pos()), x.Kind, x.Value)
            //tdAssgn = cr.Parent()
            tdPNode = cr.Parent()
            //return false
        }
    case *ast.AssignStmt:
        if x != nil {
            fmt.Printf("asgn %s: %s - %s\n", fset.Position(n.Pos()), x.Lhs, x.Rhs)
        }
        tda = *x
        tdn = n
		/*case *ast.Ident:
            if x != nil && "tests" == x.Name {
                fmt.Printf("id %s: %s\n", fset.Position(n.Pos()), x.Name)
            } */
	}
	return true
    
    /*
    p, ok := cr.Parent().(*ast.BinaryExpr)
    if !ok {
        return true
    }
    //ast.Print(nil, p)
    fmt.Println(cr.Name())
    fmt.Println(p.Op)

    /*if p.Op == token.ADD {
        cr.Replace(&ast.BasicLit{
            Kind:  token.INT,
            Value: "2",
        })
        return false
    }
    */
}, nil)
fmt.Println("whoa")
fmt.Println(tdPNode)
fmt.Printf("%T, %s", tdPNode, tdPNode)
//ast.Print(nil, fexp)

asgn 2:24: [tests] - [%!s(*ast.CompositeLit=&{0xc000833920 140 [0xc000837700 0xc000837780 0xc000837800 0xc0008378c0 0xc000837940 0xc0008379c0 0xc000837a40 0xc000837ac0 0xc000837b40 0xc000837bc0 0xc000837c40 0xc000837cc0 0xc000837d40] 941 false})]
bl 63:1:	STRING - "parse queen"
asgn 75:32: [got] - [%!s(*ast.CallExpr=&{0xc0007cd0e0 1031 [0xc0007cd140] 0 1039})]
whoa
&{name 825 0xc0007ccd00}
*ast.KeyValueExpr, &{name %!s(token.Pos=825) %!s(*ast.BasicLit=&{827 9 "parse queen"})}

87 <nil>

In [12]:
astutil.Apply(fexp, func(cr *astutil.Cursor) bool {
    n := cr.Node()
    //n := cr.Parent()
    switch x := n.(type) {
    case *ast.BasicLit:
        if token.STRING == x.Kind && "\"parse queen\"" == x.Value{
            fmt.Printf("bl %s:\t%s - %s\n", fset.Position(n.Pos()), x.Kind, x.Value)
            //tdAssgn = cr.Parent()
            tdPNode = cr.Parent()
            //return false
        }
    case *ast.AssignStmt:
        fmt.Printf("asgn %s: %s - %s\n", fset.Position(n.Pos()), x.Lhs, x.Rhs)
        tda = *x
        tdn = n
    case *ast.ExprStmt:
        if x != nil {
            fmt.Printf("exprst %s: %s - %s\n", fset.Position(n.Pos()), x.X, "")
        }
	}
	return true
}, nil)

var abuf bytes.Buffer
printer.Fprint(&abuf, fset, fexp)
fmt.Println("after\n----\n", abuf.String())
//printer.Fprint(&buf, fset, fun)                                         


asgn 2:24: [tests] - [%!s(*ast.CompositeLit=&{0xc000833920 140 [0xc000837700 0xc000837780 0xc000837800 0xc0008378c0 0xc000837940 0xc0008379c0 0xc000837a40 0xc000837ac0 0xc000837b40 0xc000837bc0 0xc000837c40 0xc000837cc0 0xc000837d40] 941 false})]
bl 63:1:	STRING - "parse queen"
exprst 74:18: &{%!s(*ast.SelectorExpr=&{0xc0007ccf40 0xc0007ccf60}) %!s(token.Pos=978) [%!s(*ast.SelectorExpr=&{0xc0007ccfa0 0xc0007ccfc0}) %!s(*ast.FuncLit=&{0xc0007cd0a0 0xc0007ce2a0})] %!s(token.Pos=0) %!s(token.Pos=1134)} - 
asgn 75:32: [got] - [%!s(*ast.CallExpr=&{0xc0007cd0e0 1031 [0xc0007cd140] 0 1039})]
exprst 76:42: &{%!s(*ast.SelectorExpr=&{0xc0007cd1e0 0xc0007cd200}) %!s(token.Pos=1071) [%!s(*ast.BasicLit=&{1072 9 "ParseCard(%s) = %d, want %d"}) %!s(*ast.SelectorExpr=&{0xc0007cd260 0xc0007cd280}) got %!s(*ast.SelectorExpr=&{0xc0007cd300 0xc0007cd320})] %!s(token.Pos=0) %!s(token.Pos=1124)} - 
after
----
 func(
	_ testing.T) {
	tests := []struct {
		name	string	`json:"name"`
		card	string	`json:"card"`

1037 <nil>

In [13]:
// golang test_code extraction pseudocode
        
// 1) Inspect the AST for the full test function using ast.Inspect(): 
//       
//    find the AssignStmt with a LHS containing the "original test data []struct name" and 
//        the RHS containing a *ast.CompositeLit matching the subtest name
//        (need to sub underscores for spaces and search for both)
//    __Store__ the "original test data []struct" name, "tests AssignStmt" node, and the "subtest data" *ast.CompositeLit node
//    find the *ast.RangeStmt with X == "original test data []struct name"
//    __Store__ the "test loop range statement node"
//    __Store__ the *ast.RangeStmt.Value "new test data struct name"
//    Traverse the RangeStmt.Body - the next node should be the *ast.CallExpr for Run()
//    Traverse the Run() *ast.CallExpr to arrive at the closure passed to the Run(): *ast.FuncLit
//    __Store__ the *ast.FuncLit.Body "Run() function literal node"
//
// 2) Edit the AST for the full test function using astutil.Apply(): 
//
//    find the "tests AssignStmt" node
//      - replace it with a new struct named "new test data struct name" and assign it "subtest data"
//
//    find the "test loop range statement node"
//      - replace it with the "Run() function literal node"
type testMeta struct {
    subTName    string           // name of subtest
    origTDName  string           // original test data []struct name
    newTDName   string           // new test data struct name
    TDAssign    ast.Node         // tests AssignStmt
    subTest     ast.CompositeLit // Run() function literal node
    rangeStmt   ast.RangeStmt    // test loop range statement node
} 

metadata := testMeta{
    subTName: "parse queen",
}

fset := token.NewFileSet()
f, err := parser.ParseFile(fset, "", "package main\n" + testCodeSrc, parser.ParseComments)
if err != nil {
    log.Fatal(err)
}
fexp, err := parser.ParseExpr(testCodeSrc)

ast.Inspect(fexp, func(n ast.Node) bool {
    switch x := n.(type) {
    case *ast.RangeStmt:        
        fmt.Printf("range %s:\t%s - %s\n", fset.Position(n.Pos()), x.X, x.Body)
        // List : []ast.Stmt(len = 1)
        // - grab the value (tt), and use it to name the test struct
        //   1 : *ast.RangeStmt (Tok: :=)
        //   Key : *ast.Ident (Name: _)
        //   Value : *ast.Ident (Name: tt)
        //     Obj : *ast.Object (Kind: var, Name: tt)
        
        // Main Body: *ast.BlockStmt - List [2]: 
        // - first node: []ast.Stmt(len = 1)
        //    - *ast.RangeStmt (Tok: :=)  [[[[[----this is the thing we want to replace-----]]]]]

        // range.body.list[1]
          // contains the Run, replace it with the body of the run...
        
        // range.body.list.[]ast.Stmt(len = 1) 
           // this list contains - *ast.ExprStmt - standalone expression in a statement list
        // *ast.ExprStmt.X is an *ast.CallExpr (to run())
        //    *ast.CallExpr.Fun -> *ast.SelectorExpr 
        //        - *ast.SelectorExpr.X = identifier t (in t.Run())
        //        - *ast.SelectorExpr.Sel = Name "Run" in t.Run()
        //    *ast.CallExpr.Args is an []ast.Expr(len = 2) which is the 2 args to t.Run(tt.name, func(t *testing.T)
        //        - 0: *ast.SelectorExpr.Sel = Name "tt.name" in t.Run()
        //        - 1: *ast.FuncLit *ast.Expr
        //              - Type: *ast.FuncType
        //               Body : *ast.BlockStmt [[[[[----the thing we want to use to replace the range stmt-----]]]]]
        //                 List : []ast.Stmt(len = 0)
    case *ast.BasicLit:
        if token.STRING == x.Kind && "\"parse queen\"" == x.Value {
            fmt.Printf("bl %s:\t%s - %s\n", fset.Position(n.Pos()), x.Kind, x.Value)
            //tdAssgn = cr.Parent()
            //tdPNode = cr.Parent()
            metadata.origTDName = "parse queen"
            //return false
        }
    case *ast.AssignStmt:
        if metadata.TDAssign == nil {
            //[TODO - search and assign]
            fmt.Printf("asgn %s: %s - %s\n", fset.Position(n.Pos()), x.Lhs, x.Rhs)
            metadata.TDAssign = n
        }
//    find the AssignStmt with a LHS containing the "original test data []struct name" and 
//        the RHS containing a *ast.CompositeLit matching the subtest name
//        (need to sub underscores for spaces and search for both)
//    __Store__ the "original test data []struct" name, "tests AssignStmt" node, and the "subtest data" *ast.CompositeLit node

    //case *ast.ExprStmt:
     //   fmt.Printf("exprst %s: %s - %s\n", fset.Position(n.Pos()), x.X, "")
    //case *ast.RangeStmt:
        //fmt.Printf("exprst %s: %s - %s\n", fset.Position(n.Pos()), x.X, "")
    }
    return true
})

fmt.Println("-----\n")
fmt.Printf("%+v\n", metadata)
/*var abuf bytes.Buffer
printer.Fprint(&abuf, fset, fun)
fmt.Println("after\n----\n", buf.String())
*/

asgn 2:24: [tests] - [%!s(*ast.CompositeLit=&{0xc00075bbc0 140 [0xc00075f500 0xc00075f580 0xc00075f600 0xc00075f6c0 0xc00075f740 0xc00075f7c0 0xc00075f840 0xc00075f8c0 0xc00075f940 0xc00075f9c0 0xc00075fa40 0xc00075fac0 0xc00075fb40] 941 false})]
bl 63:1:	STRING - "parse queen"
range 71:10:	tests - &{%!s(token.Pos=969) [%!s(*ast.ExprStmt=&{0xc00075fdc0})] %!s(token.Pos=1137)}
-----

{𒀸subTName:parse queen 𒀸origTDName:parse queen 𒀸newTDName: TDAssign:0xc00075fbc0 𒀸subTest:{Type:<nil> Lbrace:0 Elts:[] Rbrace:0 Incomplete:false} 𒀸rangeStmt:{For:0 Key:<nil> Value:<nil> TokPos:0 Tok:ILLEGAL X:<nil> Body:<nil>}}


243 <nil>

In [173]:
fset := token.NewFileSet()
f, err := parser.ParseFile(fset, "", "package main\n" + testCodeSrc, parser.ParseComments)
if err != nil {
    log.Fatal(err)
}
fexp, err := parser.ParseExpr(testCodeSrc)

//fmt.Printf("%s\n\n----\n\n", testCodeSrc)

type testMeta struct {
    subTName    string             // name of subtest
    origTDName  string             // original test data []struct name
    newTDName   string             // new test data struct name
    //TD          ast.Expr          // original tests data node
    TD          ast.CompositeLit
    //TDAssign    ast.Node         // tests AssignStmt
    subTest     ast.CompositeLit   // Run() function literal node
    rangeStmt   ast.RangeStmt      // test loop range statement node
} 

metadata := testMeta{
    subTName: "parse queen",
}

fAST, ok := f.Decls[0].(*ast.FuncDecl)
if !ok {
    fmt.Println("subtest does not contain a function as the first declaration") 
}
fbAST := fAST.Body.List  // f.Decls[0].Body.List
if 2 != len(fbAST) {
    fmt.Println("subtests are only allowed top level nodes") 
}
tdast, ok := fbAST[0].(*ast.AssignStmt) // f.Decls[0].Body.List[0]
if !ok {
    fmt.Println("subtest does not contain a test data assignment as the first node") 
}
rast, ok := fbAST[1].(*ast.RangeStmt)
if !ok {
    fmt.Println("subtest does not contain a range keyword as the second node") 
}

lhs, ok := tdast.Lhs[0].(*ast.Ident) // f.Decls[0].Body.List[0].Lhs[0]
if !ok {
    fmt.Println("subtest test data assignment not found") 
}
if ast.Var != lhs.Obj.Kind {
    fmt.Println("subtest test data assignment not a var") 
}
metadata.origTDName = lhs.Name

rhs, ok := tdast.Rhs[0].(*ast.CompositeLit) // f.Decls[0].Body.List[0].Rhs[0]
if !ok {
    fmt.Println("subtest test data assignment not a composite literal") 
}
// Loop for all of the test data structs
for i, td := range rhs.Elts {
    vals, ok := td.(*ast.CompositeLit)
    if ok {
        // Loop for all of the elements in each struct
        for _, tv := range vals.Elts {
            if kv, ok := tv.(*ast.KeyValueExpr); ok {
                if value, ok := kv.Value.(*ast.BasicLit); ok {
                    altSubTName := strings.Replace(metadata.subTName, " ", "_", -1)
                    if token.STRING == value.Kind && (strconv.Quote(metadata.subTName) == value.Value || strconv.Quote(altSubTName) == value.Value) {
                        // We have identifed the test data struct value
                        //fmt.Printf("got val of type %T -> %s\n", value, value.Value)
                        //tv is the node we want, it's a *ast.CompositeLit
                        metadata.TD = *vals
                        //value.Value = "whoaa"
                        break
                        //if key, ok := kv.Key.(*ast.Ident); ok {
                        //    fmt.Printf("got key of type %T -> %s:%s\n", kv, key, key)
                        //}
                    }
                }
            }
            
//rhs = &metadata.TD
            /*
var buf bytes.Buffer
buf.Reset()
if err := format.Node(&buf, fset, f); err != nil {
    panic(err)
}
fmt.Printf("type %T: %s", rhs, buf.String())*/

            
            //strconv.Quote("something")
            // bl is an ast.BasicLit            

            // fmt.Printf("got val of type %T -> %s:%s\n", val, val) // bl.Value) 

            //fmt.Printf("got wtf of type %T -> %s:%s\n", bl.Value, bl.Value,"wtf") // bl.Value) 
            /*altSubTName := strings.Replace(metadata.subTName, " ", "_", -1)
            if ok && (fmt.Sprintf(`%s`, metadata.subTName) == kv.Value || fmt.Sprintf("%s", altSubTName) == kv.Value) {
                fmt.Printf("got struct of type %T: %s\n", kv.Value, kv.Value) 
            } else {
                fmt.Printf("got wtf of type %T: %s\n", kv.Value, kv.Value) 
                //kvv, ok := kv.Value.(*ast.BasicLit)
                //fmt.Printf("got wtf of value %T: %s\n", fmt.Sprintf(kvv.Value, kvv.Value))
            }
            */

        }
    }
    //if metadata.TestName == td.(*ast.CompositeLit).
}

//    __Store__ the "original test data []struct" name, "tests AssignStmt" node, and the "subtest data" *ast.CompositeLit node

//rhs := tdast.Rhs


        // range.body.list.[]ast.Stmt(len = 1) 
           // this list contains - *ast.ExprStmt - standalone expression in a statement list
        // *ast.ExprStmt.X is an *ast.CallExpr (to run())
        //    *ast.CallExpr.Fun -> *ast.SelectorExpr 
        //        - *ast.SelectorExpr.X = identifier t (in t.Run())
        //        - *ast.SelectorExpr.Sel = Name "Run" in t.Run()
        //    *ast.CallExpr.Args is an []ast.Expr(len = 2) which is the 2 args to t.Run(tt.name, func(t *testing.T)
        //        - 0: *ast.SelectorExpr.Sel = Name "tt.name" in t.Run()
        //        - 1: *ast.FuncLit *ast.Expr
        //              - Type: *ast.FuncType
        //               Body : *ast.BlockStmt [[[[[----the thing we want to use to replace the range stmt-----]]]]]
        //                 List : []ast.Stmt(len = 0)

fmt.Printf("%+v\n--\n", metadata)
//fmt.Printf("%T | %+v\n", rhs.Elts, rhs.Elts)

var buf bytes.Buffer
//fmt.Printf("type %T: %s", metadata.TD, buf.String())
if err := format.Node(&buf, fset, f); err != nil {
    panic(err)
}
fmt.Printf("type %T: %s", rhs, buf.String())
buf.Reset()

//printer.Fprint(&buf, fset, rangeAST)
//fmt.Printf("type %T: %s", rangeAST, buf.String())

{𒀸subTName:parse queen 𒀸origTDName:tests 𒀸newTDName: TD:{Type:<nil> Lbrace:829 Elts:[0xc0008dc000 0xc0008dc030 0xc0008dc060] Rbrace:888 Incomplete:false} 𒀸subTest:{Type:<nil> Lbrace:0 Elts:[] Rbrace:0 Incomplete:false} 𒀸rangeStmt:{For:0 Key:<nil> Value:<nil> TokPos:0 Tok:ILLEGAL X:<nil> Body:<nil>}}
--
type *ast.CompositeLit: package main

func TestParseCard(t *testing.T) {
	tests := []struct {
		name string `json:"name"`
		card string `json:"card"`
		want int    `json:"want"`
	}{
		{
			name: "parse ace",
			card: "ace",
			want: 11,
		},
		{
			name: "parse two",
			card: "two",
			want: 2,
		},
		{
			name: "parse three",
			card: "three",
			want: 3,
		},
		{
			name: "parse four",
			card: "four",
			want: 4,
		},
		{
			name: "parse five",
			card: "five",
			want: 5,
		},
		{
			name: "parse six",
			card: "six",
			want: 6,
		},
		{
			name: "parse seven",
			card: "seven",
			want: 7,
		},
		{
			name: "parse eight",
			card: "eight",
			want: 8,
		},
		{
			name: "parse nine"